In [1]:
import sys
sys.path.append("C:/Users/Jia-Wei Chang/hu_file/LeNet5")
#自定義資料取得
import ESC50_puliic_getDataset as epg

#資料處理套件
import numpy as np
#畫圖套件
import matplotlib.pyplot as plt
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

In [2]:
#LeNet5模型
class LeNet(nn.Module):
    def __init__(self, nums_class=10):
        super(LeNet,self).__init__()
        
        #卷基層
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        
        #全連接層
        self.fullconn_layer = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(32,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,nums_class)           
        )
        
    def forward(self,x):
        output = self.conv_layer(x)
        output = self.fullconn_layer(output)
        return output


In [3]:
#測試模型對於電鋸資料集的準確度
def test_model(inputs_shape, test_loader):
    #固定模型不改變權重
    model.eval() 
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.view(inputs_shape)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += 1
            correct += (predicted == labels).sum().item()

        print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
        return 100 * correct / total

#訓練模型
def train_model(model, loss_func, optimizer, input_shape, num_epochs, train_loader, val_loader, test_loader1, test_loader3, test_loader5):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    val_loss_array = []
    val_accuracy_array = []
    test1_accuracy_array = []
    test3_accuracy_array = []
    test5_accuracy_array = []
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #包裝參數 重組形狀
            train = Variable(inputs.view(input_shape))
            labels = Variable(labels)
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(train)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #驗證準確度
        correct_val= 0
        #驗證資料總筆數
        total_val = 0
        
        for data in val_loader:
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #包裝參數 重組形狀
            val = Variable(inputs.view(input_shape))
            #向前傳播
            outputs = model(val)
            #計算loss
            val_loss = loss_func(outputs, labels)
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_val += len(labels)
            #計算判斷正確的筆數
            correct_val += (predicted == labels).float().sum()
        #計算這次驗證的準確度
        val_accuracy = 100 * correct_val / float(total_val)
        #紀錄這次驗證的準確度
        val_accuracy_array.append(val_accuracy)
        #紀錄這次驗證的loss
        val_loss_array.append(val_loss.data)
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy, val_loss.data, val_accuracy))
        #調度器走一步
        scheduler.step()
        #測試電鋸資料集 1S 3S 5S
        test1_accuracy_array.append(test_model((-1,1,128,87), test_loader1))
        test3_accuracy_array.append(test_model((-1,1,128,259), test_loader3))
        test5_accuracy_array.append(test_model((-1,1,128,431), test_loader5))
    return train_loss_array, train_accuracy_array, val_loss_array, val_accuracy_array, test1_accuracy_array, test3_accuracy_array, test5_accuracy_array

In [4]:
train_loss_array = []
train_accuracy_array = []
val_loss_array = []
val_accuracy_array = []
test1_accuracy_array = []
test3_accuracy_array = []
test5_accuracy_array = []
#----------------------------------
#訓練相關參數
#指定訓練位置
device = torch.device('cuda:0')
#訓練次數
num_epochs = 30
#聲音長度
sound = [1, 3, 5]
#批次大小
BATCH_SIZE = 64
#分類數量
num_class = 10
#學習率
LR = 0.0002
#輸入形狀
input_shape = [(-1,1,128,87), (-1,1,128,259), (-1,1,128,431)]

#----------------------------------
#設定存檔位置
f = open("train.txt", "w")
#k-fold訓練
for fold in range(1,6):
    train_loss_TL = []
    train_accuracy_TL = []
    val_loss_TL = []
    val_accuracy_TL = []
    test1_accuracy_TL = []
    test3_accuracy_TL = []
    test5_accuracy_TL = []
    #建立模型
    model = LeNet(nums_class = num_class)
    #指定訓練位置
    model.to(device)
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    
    #資料抓入
    test_loader1 = epg.getTestDataset(sound[0])
    test_loader3 = epg.getTestDataset(sound[1])
    test_loader5 = epg.getTestDataset(sound[2])
    
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("TL: ", TL+1, "fold: ", fold)
        print("----------------------------------------------------------------------------------------------------------------")
        #資料抓入
        train, val = epg.getTrainDataset(fold, sound[TL], num_class)
        #分批方法
        train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = True)
        val_loader = torch.utils.data.DataLoader(val, batch_size = BATCH_SIZE, shuffle = True)
        
        #優化器
        optimizer = torch.optim.Adam(model.parameters(), lr=LR)
        #調度器
        scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
        #訓練模型
        train_loss, train_accuracy, val_loss, val_accuracy, test1_accuracy, test3_accuracy, test5_accuracy = train_model(model, loss_func, optimizer, input_shape[TL], num_epochs, train_loader, val_loader, test_loader1, test_loader3, test_loader5)
    
        #將gpu資料取回
        train_loss = torch.stack(train_loss,dim=-1).cpu()
        train_accuracy = torch.stack(train_accuracy,dim=-1).cpu()
        val_loss = torch.stack(val_loss,dim=-1).cpu()
        val_accuracy = torch.stack(val_accuracy,dim=-1).cpu()
    
        for i in range(num_epochs):
            # 寫入資料
            f.write("TL :{} fold: {} Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(TL+1 ,fold, i+1, num_epochs, train_loss[i], train_accuracy[i], val_loss[i], val_accuracy[i], test1_accuracy[i], test3_accuracy[i], test5_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        train_loss_TL.append(train_loss)
        train_accuracy_TL.append(train_accuracy)
        val_loss_TL.append(val_loss)
        val_accuracy_TL.append(val_accuracy)
        test1_accuracy_TL.append(test1_accuracy)
        test3_accuracy_TL.append(test3_accuracy)
        test5_accuracy_TL.append(test5_accuracy)
    modelName = "Fold_" + str(fold) + "_TF.pkl"
    torch.save(model, modelName)
    train_loss_array.append(train_loss_TL)
    train_accuracy_array.append(train_accuracy_TL)
    val_loss_array.append(val_loss_TL)
    val_accuracy_array.append(val_accuracy_TL)
    test1_accuracy_array.append(test1_accuracy_TL)
    test3_accuracy_array.append(test3_accuracy_TL)
    test5_accuracy_array.append(test5_accuracy_TL)
f.close()

----------------------------------------------------------------------------------------------------------------
TL:  1 fold:  1
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/30 Train_Loss: 2.039940595626831 Train_acc: 12.812500% Val_Loss: 1.8993862867355347 Val_accuracy: 11.750000%
Test accuracy of the model: 49.200000%
Test accuracy of the model: 36.000000%
Test accuracy of the model: 30.000000%
Train Epoch: 2/30 Train_Loss: 2.052135705947876 Train_acc: 22.687500% Val_Loss: 1.9508126974105835 Val_accuracy: 22.750000%
Test accuracy of the model: 67.400000%
Test accuracy of the model: 68.333333%
Test accuracy of the model: 68.000000%
Train Epoch: 3/30 Train_Loss: 2.0199079513549805 Train_acc: 24.562500% Val_Loss: 1.839119553565979 Val_accuracy: 25.250000%
Test accuracy of the model: 75.800000%
Test accuracy of the model: 79.333333%
Test accuracy of the model: 83.000000%
Train Epoch: 4/30 Train_Loss: 1.730

Test accuracy of the model: 80.000000%
Train Epoch: 3/30 Train_Loss: 1.224854588508606 Train_acc: 48.229168% Val_Loss: 1.4736555814743042 Val_accuracy: 42.083336%
Test accuracy of the model: 74.600000%
Test accuracy of the model: 78.333333%
Test accuracy of the model: 81.000000%
Train Epoch: 4/30 Train_Loss: 1.279457926750183 Train_acc: 48.645836% Val_Loss: 1.7043331861495972 Val_accuracy: 37.083336%
Test accuracy of the model: 76.000000%
Test accuracy of the model: 79.666667%
Test accuracy of the model: 82.000000%
Train Epoch: 5/30 Train_Loss: 1.2055144309997559 Train_acc: 50.520836% Val_Loss: 1.519235610961914 Val_accuracy: 37.916668%
Test accuracy of the model: 75.800000%
Test accuracy of the model: 79.333333%
Test accuracy of the model: 82.000000%
Train Epoch: 6/30 Train_Loss: 1.1365816593170166 Train_acc: 49.375004% Val_Loss: 1.3018592596054077 Val_accuracy: 38.333336%
Test accuracy of the model: 75.800000%
Test accuracy of the model: 79.000000%
Test accuracy of the model: 82.0000

Test accuracy of the model: 90.000000%
Train Epoch: 6/30 Train_Loss: 1.2422287464141846 Train_acc: 56.562500% Val_Loss: 1.2689777612686157 Val_accuracy: 50.000000%
Test accuracy of the model: 79.400000%
Test accuracy of the model: 85.333333%
Test accuracy of the model: 87.000000%
Train Epoch: 7/30 Train_Loss: 1.1263494491577148 Train_acc: 59.062500% Val_Loss: 1.482221245765686 Val_accuracy: 47.500000%
Test accuracy of the model: 82.000000%
Test accuracy of the model: 87.333333%
Test accuracy of the model: 90.000000%
Train Epoch: 8/30 Train_Loss: 1.0654242038726807 Train_acc: 56.875000% Val_Loss: 1.1388161182403564 Val_accuracy: 52.500000%
Test accuracy of the model: 83.000000%
Test accuracy of the model: 88.666667%
Test accuracy of the model: 91.000000%
Train Epoch: 9/30 Train_Loss: 1.185507893562317 Train_acc: 57.500000% Val_Loss: 1.3203625679016113 Val_accuracy: 48.750000%
Test accuracy of the model: 85.000000%
Test accuracy of the model: 90.333333%
Test accuracy of the model: 92.000

Test accuracy of the model: 63.000000%
Train Epoch: 9/30 Train_Loss: 1.6684248447418213 Train_acc: 34.625000% Val_Loss: 1.640733003616333 Val_accuracy: 35.750000%
Test accuracy of the model: 68.800000%
Test accuracy of the model: 69.666667%
Test accuracy of the model: 68.000000%
Train Epoch: 10/30 Train_Loss: 1.767396330833435 Train_acc: 34.312500% Val_Loss: 1.9518836736679077 Val_accuracy: 34.000000%
Test accuracy of the model: 75.400000%
Test accuracy of the model: 80.000000%
Test accuracy of the model: 82.000000%
Train Epoch: 11/30 Train_Loss: 1.81242036819458 Train_acc: 37.062500% Val_Loss: 1.8633575439453125 Val_accuracy: 36.500000%
Test accuracy of the model: 68.200000%
Test accuracy of the model: 72.333333%
Test accuracy of the model: 69.000000%
Train Epoch: 12/30 Train_Loss: 1.6971385478973389 Train_acc: 37.000000% Val_Loss: 1.881567358970642 Val_accuracy: 33.500000%
Test accuracy of the model: 67.800000%
Test accuracy of the model: 71.000000%
Test accuracy of the model: 69.000

Test accuracy of the model: 89.000000%
Train Epoch: 12/30 Train_Loss: 1.2374980449676514 Train_acc: 48.750004% Val_Loss: 1.3675960302352905 Val_accuracy: 47.500004%
Test accuracy of the model: 78.800000%
Test accuracy of the model: 83.666667%
Test accuracy of the model: 86.000000%
Train Epoch: 13/30 Train_Loss: 1.3819096088409424 Train_acc: 49.270836% Val_Loss: 1.5934089422225952 Val_accuracy: 47.083336%
Test accuracy of the model: 72.400000%
Test accuracy of the model: 77.666667%
Test accuracy of the model: 78.000000%
Train Epoch: 14/30 Train_Loss: 1.1270496845245361 Train_acc: 49.895836% Val_Loss: 1.2652723789215088 Val_accuracy: 50.000004%
Test accuracy of the model: 78.600000%
Test accuracy of the model: 85.666667%
Test accuracy of the model: 89.000000%
Train Epoch: 15/30 Train_Loss: 1.2766200304031372 Train_acc: 50.208336% Val_Loss: 1.2404698133468628 Val_accuracy: 51.250004%
Test accuracy of the model: 79.800000%
Test accuracy of the model: 86.333333%
Test accuracy of the model: 

...

Train Epoch: 27/30 Train_Loss: 1.6260876655578613 Train_acc: 41.187500% Val_Loss: 1.9540127515792847 Val_accuracy: 41.500000%
Test accuracy of the model: 67.000000%
Test accuracy of the model: 69.000000%
Test accuracy of the model: 67.000000%
Train Epoch: 28/30 Train_Loss: 1.4634712934494019 Train_acc: 42.125000% Val_Loss: 1.5843007564544678 Val_accuracy: 38.500000%
Test accuracy of the model: 65.600000%
Test accuracy of the model: 65.666667%
Test accuracy of the model: 65.000000%
Train Epoch: 29/30 Train_Loss: 1.579209566116333 Train_acc: 41.812500% Val_Loss: 1.3330806493759155 Val_accuracy: 40.750000%
Test accuracy of the model: 69.200000%
Test accuracy of the model: 72.666667%
Test accuracy of the model: 70.000000%
Train Epoch: 30/30 Train_Loss: 1.4508938789367676 Train_acc: 40.937500% Val_Loss: 1.3188689947128296 Val_accuracy: 40.000000%
Test accuracy of the model: 67.200000%
Test accuracy of the model: 69.666667%
Test accuracy of the model: 67.000000%
-----------------------------

Test accuracy of the model: 91.000000%
Train Epoch: 30/30 Train_Loss: 1.124825119972229 Train_acc: 52.395836% Val_Loss: 1.1064718961715698 Val_accuracy: 52.083336%
Test accuracy of the model: 80.000000%
Test accuracy of the model: 86.666667%
Test accuracy of the model: 91.000000%
----------------------------------------------------------------------------------------------------------------
TL:  3 fold:  4
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/30 Train_Loss: 1.4817227125167847 Train_acc: 46.562500% Val_Loss: 1.392001986503601 Val_accuracy: 42.500000%
Test accuracy of the model: 77.400000%
Test accuracy of the model: 80.333333%
Test accuracy of the model: 83.000000%
Train Epoch: 2/30 Train_Loss: 1.261390209197998 Train_acc: 44.375000% Val_Loss: 1.4333624839782715 Val_accuracy: 56.250000%
Test accuracy of the model: 80.800000%
Test accuracy of the model: 87.666667%
Test accuracy of the model: 92.000

Test accuracy of the model: 0.000000%
Train Epoch: 2/30 Train_Loss: 2.149463653564453 Train_acc: 11.437500% Val_Loss: 2.0425729751586914 Val_accuracy: 18.500000%
Test accuracy of the model: 55.800000%
Test accuracy of the model: 46.666667%
Test accuracy of the model: 42.000000%
Train Epoch: 3/30 Train_Loss: 2.0161304473876953 Train_acc: 19.375000% Val_Loss: 2.0455760955810547 Val_accuracy: 26.000000%
Test accuracy of the model: 78.000000%
Test accuracy of the model: 81.000000%
Test accuracy of the model: 83.000000%
Train Epoch: 4/30 Train_Loss: 1.9340940713882446 Train_acc: 28.062500% Val_Loss: 1.9983606338500977 Val_accuracy: 24.250000%
Test accuracy of the model: 83.800000%
Test accuracy of the model: 90.000000%
Test accuracy of the model: 92.000000%
Train Epoch: 5/30 Train_Loss: 1.9105290174484253 Train_acc: 29.812500% Val_Loss: 1.8344106674194336 Val_accuracy: 26.000000%
Test accuracy of the model: 75.200000%
Test accuracy of the model: 79.000000%
Test accuracy of the model: 83.000

Test accuracy of the model: 92.000000%
Train Epoch: 5/30 Train_Loss: 1.3176007270812988 Train_acc: 46.041668% Val_Loss: 1.4895124435424805 Val_accuracy: 41.250004%
Test accuracy of the model: 79.400000%
Test accuracy of the model: 85.333333%
Test accuracy of the model: 88.000000%
Train Epoch: 6/30 Train_Loss: 1.1995071172714233 Train_acc: 47.604168% Val_Loss: 1.5065789222717285 Val_accuracy: 44.166668%
Test accuracy of the model: 80.800000%
Test accuracy of the model: 85.666667%
Test accuracy of the model: 90.000000%
Train Epoch: 7/30 Train_Loss: 1.186436414718628 Train_acc: 48.229168% Val_Loss: 1.2994388341903687 Val_accuracy: 43.333336%
Test accuracy of the model: 79.000000%
Test accuracy of the model: 83.333333%
Test accuracy of the model: 87.000000%
Train Epoch: 8/30 Train_Loss: 1.2796229124069214 Train_acc: 49.270836% Val_Loss: 1.5288692712783813 Val_accuracy: 42.916668%
Test accuracy of the model: 81.000000%
Test accuracy of the model: 87.333333%
Test accuracy of the model: 92.00

Test accuracy of the model: 90.000000%
Train Epoch: 8/30 Train_Loss: 1.0377302169799805 Train_acc: 57.812500% Val_Loss: 1.2159940004348755 Val_accuracy: 56.250000%
Test accuracy of the model: 80.800000%
Test accuracy of the model: 85.666667%
Test accuracy of the model: 89.000000%
Train Epoch: 9/30 Train_Loss: 1.0366909503936768 Train_acc: 59.062500% Val_Loss: 1.3076519966125488 Val_accuracy: 47.500000%
Test accuracy of the model: 85.800000%
Test accuracy of the model: 90.666667%
Test accuracy of the model: 94.000000%
Train Epoch: 10/30 Train_Loss: 1.0921584367752075 Train_acc: 61.562500% Val_Loss: 1.168079137802124 Val_accuracy: 55.000000%
Test accuracy of the model: 83.200000%
Test accuracy of the model: 87.666667%
Test accuracy of the model: 91.000000%
Train Epoch: 11/30 Train_Loss: 1.062828779220581 Train_acc: 60.625000% Val_Loss: 1.3490092754364014 Val_accuracy: 50.000000%
Test accuracy of the model: 84.200000%
Test accuracy of the model: 91.333333%
Test accuracy of the model: 93.0

In [5]:
#設定存檔位置
f = open("lastData.txt", "w")
for TL in range(3):
    train_loss_last = []
    train_accuracy_last = []
    val_loss_last = []
    val_accuracy_last = []
    test1_accuracy_last = []
    test3_accuracy_last = []
    test5_accuracy_last = []
    
    #整理出訓練完的數值
    for fold in range(5):
        train_loss_last.append(train_loss_array[fold][TL][num_epochs-1])
        train_accuracy_last.append(train_accuracy_array[fold][TL][num_epochs-1])
        val_loss_last.append(val_loss_array[fold][TL][num_epochs-1])
        val_accuracy_last.append(val_accuracy_array[fold][TL][num_epochs-1])
        test1_accuracy_last.append(test1_accuracy_array[fold][TL][num_epochs-1])
        test3_accuracy_last.append(test3_accuracy_array[fold][TL][num_epochs-1])
        test5_accuracy_last.append(test5_accuracy_array[fold][TL][num_epochs-1])
        # 寫入資料
        f.write("Fold: {} Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(fold+1, train_loss_array[fold][TL][num_epochs-1], train_accuracy_array[fold][TL][num_epochs-1], val_loss_array[fold][TL][num_epochs-1], val_accuracy_array[fold][TL][num_epochs-1], test1_accuracy_array[fold][TL][num_epochs-1], test3_accuracy_array[fold][TL][num_epochs-1], test5_accuracy_array[fold][TL][num_epochs-1]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    # 寫入資料
    f.write("Max Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(max(train_loss_last), max(train_accuracy_last), max(val_loss_last), max(val_accuracy_last), max(test1_accuracy_last), max(test3_accuracy_last), max(test5_accuracy_last)))
    f.write("Min Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(min(train_loss_last), min(train_accuracy_last), min(val_loss_last), min(val_accuracy_last), min(test1_accuracy_last), min(test3_accuracy_last), min(test5_accuracy_last)))
    f.write("Avg Train_Loss: {} Train_acc: {:.6f}% Val_Loss: {} Val_accuracy: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(np.mean(train_loss_last), np.mean(train_accuracy_last), np.mean(val_loss_last), np.mean(val_accuracy_last), np.mean(test1_accuracy_last), np.mean(test3_accuracy_last), np.mean(test5_accuracy_last)))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
f.close()